In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import gensim

In [2]:
df = pd.read_json("../data/processed_data/processed.json")
dti = pd.to_datetime(df[["year", "month", "day", "hour"]], errors="coerce")
df["time"] = dti
df

,paragraph,hour,day,month,year,filename,time
0,תם שימוע ראש הממשלה בינימין נתניהו לאחר עשר שע...,21,7,10,2019,/Users/sapir/PycharmProjects/hebnlp_project/da...,2019-10-07 21:00:00
1,גבר בן שלושים נורה ברחוב העלייה בלוד מצבו קשה ...,21,7,10,2019,/Users/sapir/PycharmProjects/hebnlp_project/da...,2019-10-07 21:00:00
2,הרמטכל רב אלוף אביב כוכבי מזהיר הערב לא נאפשר ...,21,7,10,2019,/Users/sapir/PycharmProjects/hebnlp_project/da...,2019-10-07 21:00:00
3,שר התחבורה בצלאל סמוטריץ הודיע בעמוד הפייסבוק ...,21,7,10,2019,/Users/sapir/PycharmProjects/hebnlp_project/da...,2019-10-07 21:00:00
4,נשיא ארצות הברית טראמפ הודיע יציב קווים אדומים...,21,7,10,2019,/Users/sapir/PycharmProjects/hebnlp_project/da...,2019-10-07 21:00:00
...,...,...,...,...,...,...,...
5141,בחירות אלפיים עשרים ואחת יושב ראש ימינה חבר הכ...,8,9,2,2021,/Users/sapir/PycharmProjects/hebnlp_project/da...,2021-02-09 08:00:00
5142,המחדל בנבטים במהלך אותרה הנקודה בגדר שממנה ככל...,8,9,2,2021,/Users/sapir/PycharmProjects/hebnlp_project/da...,2021-02-09 08:00:00
5143,הדיונים על פתיחת המסחר באמצעות הדרכון הירוק יי...,8,9,2,2021,/Users/sapir/PycharmProjects/hebnlp_project/da...,2021-02-09 08:00:00
5144,פרסום ראשון חיילת פנתה לרמטכל בדרישה פתח את כל...,8,9,2,2021,/Users/sapir/PycharmProjects/hebnlp_project/da...,2021-02-09 08:00:00


In [3]:
print(df.loc[0, "paragraph"])

תם שימוע ראש הממשלה בינימין נתניהו לאחר עשר שעות של דיונים בתיק אלף תיק המתנות היום נחתמו ארבעת ימי השימוע כעת על היועץ המשפטי לממשלה אביחי מנדלבליט להחליט האם להגיש אישום בארבעת התיקים יובל אראל מציין עורכי הדין יגישו סיכומים כתובים אחרי סוכות ומזכיר המשנה לפרקליט המדינה ליאת בן ארי נעדרה מן השימוע ביומיים האחרונים בשל חופשה משפחתית


# Tokenization

In [4]:
text_data = df.paragraph.values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(text_data))

# Bigrams and Trigrams

In [12]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=200)
trigram = gensim.models.Phrases(bigram[data_words], threshold=200) 
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [13]:
trigram.export_phrases()

{'היועץ_המשפטי': 48861.64791666667,
 'לממשלה_אביחי': 87388.51886792453,
 'היועץ_המשפטי_לממשלה_אביחי': 336693.7083333333,
 'לממשלה_אביחי_מנדלבליט': 677.4303788211204,
 'להגיש_אישום': 597.8432760364004,
 'יובל_אראל': 17206.182539682537,
 'עורכי_הדין': 1895.0865384615386,
 'בן_ארי': 2695.9618430525557,
 'ליאת_בן_ארי': 16424.083333333332,
 'גבר_בן': 246.6998576477518,
 'חובשי_מגן': 54746.944444444445,
 'דוד_אדום': 12282.149781341108,
 'חובשי_מגן_דוד_אדום': 35194.46428571429,
 'דוד_אדום_טיפלו': 365.6567717996289,
 'ופינו_אותו': 924.4324577861164,
 'למרכז_הרפואי': 51865.52631578947,
 'אסף_הרופא': 90332.45833333333,
 'הרמטכל_רב': 1305.601094470046,
 'הרמטכל_רב_אלוף': 1282.6958121109224,
 'אביב_כוכבי': 4817.731111111111,
 'אלוף_אביב_כוכבי': 3457.7017543859647,
 'רמי_שני': 4548.810982888977,
 'רמי_שני_מזכיר': 312.83968253968254,
 'המתיחות_הביטחונית': 386.4490196078431,
 'התחבורה_בצלאל': 15160.692307692309,
 'שר_התחבורה_בצלאל': 1459.9185185185186,
 'התחבורה_בצלאל_סמוטריץ': 522.7824933687002,
 'ב